This is a guidance to generate a national competititon network, with 3 division leagues and a national cup inside.
The national league is a double round robin and national cup is default oneleg.

First step, import the needed module

In [ ]:
from dfg_rating.model.network.international_network import CountryLeague
from dfg_rating.model.rating.controlled_trend_rating import ControlledRandomFunction, ControlledTrendRating
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast

Then, give each division a rating trend. Each team in this division will follow this trend.

Team's rating in season 0 day Y = starting_point + trend/*delta/*(Y-1)

When the season ends, the rating will + season_delta

In default, each division follows trends like this:

* true_rating_level1, starting_point follows normal distribution with loc=1000, scale=50
* true_rating_level2,starting_point follows normal distribution with loc=600, scale=50
* true_rating_level3,starting_point follows normal distribution with loc=300, scale=50
* delta follows normal distribution with loc=0, scale=.5
* trend follows normal distribution with loc=0, scale=.2
* season_delta follows normal distribution with loc=0, scale=30

For those teams who does not belong to top-3 divisions, their rating is defaultly the same with the 3-rd one.

In [ ]:
rating_level1 = ControlledTrendRating(
    starting_point=ControlledRandomFunction(distribution='normal', loc=800, scale=100),
    delta=ControlledRandomFunction(distribution='normal', loc=0, scale=.5),
    trend=ControlledRandomFunction(distribution='normal', loc=0, scale=.2),
    season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=30),
    rating_name='true_rating'
)

rating_level2 = ControlledTrendRating(
    starting_point=ControlledRandomFunction(distribution='normal', loc=500, scale=50),
    delta=ControlledRandomFunction(distribution='normal', loc=0, scale=.5),
    trend=ControlledRandomFunction(distribution='normal', loc=0, scale=.2),
    season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=30),
    rating_name='true_rating'
)

rating_level3 = ControlledTrendRating(
    starting_point=ControlledRandomFunction(distribution='normal', loc=200, scale=10),
    delta=ControlledRandomFunction(distribution='normal', loc=0, scale=.5),
    trend=ControlledRandomFunction(distribution='normal', loc=0, scale=.2),
    season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=30),
    rating_name='true_rating'
)

Then create the forecast method for the national network.

This is a pre-trined logisticRegression for outcome predict. For each match, it will give the match result based on the ratings of two team.

In [ ]:
forecast_test = LogFunctionForecast(
    outcomes=['home', 'draw', 'away'],
    coefficients=[-0.9, 0.3],
    beta_parameter=0.006
)

Then, config how do you want this network looks like:

* teams: total team number of this network, contains those who doesn't in top-3 leagues, these extra teams should more than the promotion_number.
* level\*_teams: number of teams in division \*, for division 2 and 3, the number should be more than twice of promotion_number.
* promotion_number: number of temas gets promoted/relegated of each division.
* prob_levelA_levelB: controls the probability of teams from division_A and B have a match. Since the national cup is one-leg default, if prob = 0.5, then 100 edges between division A and B, (100/2)*0.5=25 of them should be active.
* seasons: controls how many seasons you want simulate
* rating_mode: choose from 'keep','mix' and 'interchange'. This controls how do you want the promoted/relegated team get their start rating at the beginning of season. 

For example, team A, B, C are from division 1 with end-of-season rating 9, 10, 11. team X, Y, Z from division 2 with end-of-season rating 2, 4, 6. team A B relegated, team X Z promoted.

For 'keep' mode: A, B, X, Z still keep their end-of-season rank and their original trends.


For 'mix' mode: A, B will have rating + mean rating difference of two divisions. In this example, mean of division 1 is 10, mean rating of division 2 is 4. So A will get a 9-(10-4)=3, X will get a 2+(10-4)=8. And they will continue with new division's rating trends.

For 'interchange' mode, the mean rating of promoted teams is mean(2+6)=3. Then A and B will get a new rating that keeps new_A:new_B = A:B, also the mean(new_A, new_B)=3, which is new_A = 3/(9+10)*9. And they will continue with new division's rating trends.

Note: the new season's start rating will always + season_delta

other config you can add:
* days_between_rounds: controls the shcdule of the competition.


In [ ]:
test_network = CountryLeague(
    teams=20,
    level1_teams=5,
    level2_teams=5,
    level3_teams=5,
    promotion_number=2,
    prob_level1_level2=0.10,
    prob_level1_level3=0.05,
    prob_level2_level3=0.00,
    true_rating_level1=rating_level1,
    true_rating_level2=rating_level2,
    true_rating_level3=rating_level3,
    true_forecast=forecast_test,
    seasons=5,
    rating_mode='interchange'
)

Then you can export or visulize the network.

In [ ]:
test_network.export(ratings=['true_rating','ranking'],filename='test_national_network.csv')

import dfg_rating.viz.jupyter_widgets as DFGWidgets
# display network explorer
app = DFGWidgets.NetworkExplorer(
    network=test_network,
    edge_props=["round"]
)
app.run('internal', debug=True, port=8001)